# LLM Reasoning Experiments

The experiment below shows the non-determinstic nature of LLMs. It also shows how reasoning can help it converge
to the right answer. This can also be used to confirm the "Consistency" principle outlined in Lecture 1.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## Few-shot prompting and no reasoning
Run this to see how gpt-4o produces the incorrect answer using just few-shot (2 examples) prompting. Note that running
this on ChatGPT might produce different results since that model has "reasoning" based prompting logic built into its system message as a default.

In [11]:
from openai import OpenAI
client = OpenAI()

# https://platform.openai.com/docs/api-reference/chat/create

USER_QUERY = """
Q: “Elon Musk” A: “nk”
Q: “Bill Gates” A: “ls”
Q: “Barack Obama” A: ?
"""


def run_user_query_oai_model(model_str, user_query):
    completion = client.chat.completions.create(
        model=model_str,
        messages=[
            {"role": "system", "content": "You are a helpful AI Assistant."},
            {
                "role": "user",
                "name": "Joe",
                "content": user_query
            }
        ]
    )
    return completion.choices[0].message
    
def run_user_query_oai(user_query):
    return run_user_query_oai_model("gpt-4o", user_query)

for i in range(1, 11):
    print("\n\n------------------------------------")
    print("Response:", run_user_query_oai(USER_QUERY).content)



------------------------------------
Response: “ck”


------------------------------------
Response: “ac”


------------------------------------
Response: “ck”


------------------------------------
Response: “ck”


------------------------------------
Response: The answer is "ac". The pattern in the examples involves taking the last two letters of the first name and the last two letters of the last name, and moving to the previous letters in the alphabet for each pair. For "Barack Obama":

- From "ac" in "Barack", the previous two letters are still "ac".
- From "ma" in "Obama", the previous two letters are still "ma".

So the answer remains "ac".


------------------------------------
Response: The pattern involves taking the last two letters of the first name and the last two letters of the last name. For "Barack Obama," the last two letters of the first name are "ck" and the last two letters of the last name are "ma." Therefore, the answer is "ckma."


----------------------------

# Chain of thought with reasoning

In [6]:
USER_QUERY_WITH_CHAIN_OF_THOUGHT = """
Q: “Elon Musk” A: “nk”
Q: “Bill Gates” A: “ls”
Q: “Barack Obama” A: ?
Provide a reason and arrive at the solution step by step.
"""
print("Response:", run_user_query_oai(USER_QUERY_WITH_CHAIN_OF_THOUGHT).content)
# Prints the correct answer. 

Response: ChatCompletionMessage(content='To solve this puzzle, we need to look at the pattern in how each name is transformed into an answer based on the examples given.\n\nLet\'s analyze the transformations:\n\n1. **Elon Musk** becomes **nk**:\n   - We take the last letter of the first name "Elon" which is "n".\n   - We take the last letter of the last name "Musk" which is "k".\n   - Combine these letters, and we get "nk".\n\n2. **Bill Gates** becomes **ls**:\n   - We take the last letter of the first name "Bill" which is "l".\n   - We take the last letter of the last name "Gates" which is "s".\n   - Combine these letters, and we get "ls".\n\nFollowing this pattern:\n\n3. **Barack Obama**:\n   - Take the last letter of the first name "Barack" which is "k".\n   - Take the last letter of the last name "Obama" which is "a".\n   - Combine these letters, and we get "ka".\n\nTherefore, the answer for "Barack Obama" is "ka".', refusal=None, role='assistant', function_call=None, tool_calls=No

In [22]:
# Lets see if it thinks step by step.
COT_WITH_EXAMPLES = """
Q: A juggler can juggle 16 balls. Half of the balls are golf balls, and half of the golfs balls are blue. How many blue golf balls are there ?
A: The answer (arabic numerals) is
"""
print("Response:", run_user_query_oai_model("gpt-4o", COT_WITH_EXAMPLES).content)

Response: To solve this problem, we need to first determine the number of golf balls the juggler is juggling. Since the juggler has 16 balls in total and half of them are golf balls:

\[
\text{Number of golf balls} = \frac{16}{2} = 8
\]

Now, since half of the golf balls are blue, we calculate the number of blue golf balls:

\[
\text{Number of blue golf balls} = \frac{8}{2} = 4
\]

Therefore, the number of blue golf balls is 4.

The answer (in Arabic numerals) is 4.


In [23]:
# Self consistency Example 
SELF_CONSISTENCY_PROMPT = """
Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four.
She sells the remainder for $2 per egg. How much does she make every day ?
"""
print("Response:", run_user_query_oai_model("gpt-4o", SELF_CONSISTENCY_PROMPT).content)

Response: Janet's ducks lay 16 eggs per day. She uses 3 eggs for breakfast and 4 eggs for baking muffins, which totals 7 eggs each day. 

So, the number of eggs she has left to sell each day is:

\[ 16 - 7 = 9 \text{ eggs} \]

She sells these 9 eggs at $2 per egg, so her earnings each day are:

\[ 9 \times 2 = 18 \text{ dollars} \]

Janet makes $18 every day from selling the eggs.


In [29]:
# Irrelevant context example, how irrelevant context can cause LLMs to err. Correct answer is $76 
# GPT-3.5-turbo makes more errors than gpt-4o

IRRELEVANT_CONTEXT_PROMPT = """
Lucy has $65 in the bank. She made a $15 deposit and then followed by a $4 withdrawal. Maria's monthly rent is $10. And another $3 for utilities for Maria. What is Lucky's bank balance?
"""
print("Response:", run_user_query_oai_model("gpt-3.5-turbo", IRRELEVANT_CONTEXT_PROMPT).content)

Response: To calculate Lucy's bank balance after the transactions, we start with the initial amount she had in the bank, which is $65. 

Lucy made a $15 deposit, so we add $15 to her balance: $65 + $15 = $80

Then she made a $4 withdrawal, so we subtract $4 from her balance: $80 - $4 = $76

Next, we consider Maria's expenses for the month. Maria's rent is $10, and utilities cost $3. So, we subtract these expenses from Lucy's balance: $76 - $10 - $3 = $63

Therefore, after all the transactions and expenses, Lucy's bank balance is $63.
